In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.activations import sigmoid, relu
from lab_coffee_utils import load_coffee_data

In [11]:
X, y = load_coffee_data()
print(X.shape, y.shape)

(200, 2) (200, 1)


In [15]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


In [17]:
Xt = np.tile(Xn, (1000, 1))
Yt = np.tile(y, (1000, 1))
print(Xt.shape, Yt.shape)

(200000, 2) (200000, 1)


In [19]:
tf.random.set_seed(1234)
model = Sequential(
  [
    tf.keras.Input(shape=(2,)),
    Dense(3, activation="sigmoid", name="layer1"),
    Dense(1, activation="sigmoid", name="layer2")
  ]
)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(
  loss = tf.keras.losses.BinaryCrossentropy(),
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
)
model.fit(Xt, Yt, epochs=10)

Epoch 1/10
6250/6250 [==============================] - 2s 329us/step - loss: 0.0023
Epoch 2/10
6250/6250 [==============================] - 2s 322us/step - loss: 0.0017
Epoch 3/10
6250/6250 [==============================] - 2s 326us/step - loss: 0.0013
Epoch 4/10
6250/6250 [==============================] - 2s 319us/step - loss: 9.6675e-04
Epoch 5/10
6250/6250 [==============================] - 2s 320us/step - loss: 7.4972e-04
Epoch 6/10
6250/6250 [==============================] - 2s 324us/step - loss: 5.8506e-04
Epoch 7/10
6250/6250 [==============================] - 2s 322us/step - loss: 4.5989e-04
Epoch 8/10
6250/6250 [==============================] - 2s 322us/step - loss: 3.7428e-04
Epoch 9/10
6250/6250 [==============================] - 2s 320us/step - loss: 2.9597e-04
Epoch 10/10
6250/6250 [==============================] - 2s 323us/step - loss: 2.6340e-04


In [26]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(W1, b1)
print(W2, b2)

[[ -0.20564644 -12.211443    16.231604  ]
 [ -9.464983    -0.27798384  13.467447  ]] [-11.693058  -13.16497     2.2666667]
[[-72.39911]
 [-76.81499]
 [-62.92379]] [43.12784]


In [28]:
X_test = np.array([
  [200, 13.9],
  [200, 17]
])
X_testn = norm_l(X_test)
prediction = model.predict(X_testn)
print(prediction)

1/1 [==============================] - 0s 15ms/step
[[9.9922967e-01]
 [2.4817723e-09]]


In [30]:
yhat = np.zeros_like(prediction)
for i in range(len(prediction)):
  if prediction[i] >= 0.5:
    yhat[i] = 1
  else:
    yhat[i] = 0
print(yhat)

[[1.]
 [0.]]
